In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import kaleido


DATA = Path("C:\\Users\\lamia\\Downloads\\week2-data-work---\\data\\processed\\analytics_table.parquet")
ROOT = Path(r"C:\Users\lamia\Downloads\week2-data-work---")
FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2):
   path.parent.mkdir(parents=True, exist_ok=True)
   fig.write_image(str(path), scale=scale)

df = pd.read_parquet(DATA)

df

print("Rows:", len(df))
print("Cols:", len(df.columns))
print(df.dtypes.head(15))
print(df.isna().sum().sort_values(ascending=False).head(10))

Rows: 5
Cols: 19
order_id              string[python]
user_id               string[python]
amount                       Float64
quantity                       Int64
created_at       datetime64[ns, UTC]
status                        object
status_norm           string[python]
status_clean                  object
amount_isna                     bool
quantity_isna                   bool
date                          object
year                         float64
month                 string[python]
dow                           object
hour                         float64
dtype: object
amount                1
created_at            1
quantity              1
amount_winsor         1
dow                   1
date                  1
amount__is_outlier    1
hour                  1
year                  1
month                 1
dtype: int64
